### 高校大数据，扑克牌组合优化等级标注

##### 训练数据集包括11个字段，字段的含义如下所示，牌面花色用C,D,H,S表示，分表代表梅花、方块、红桃和黑桃，牌面大小用1-10以及J、Q、K来表示。需要注意，字段11是每条数据的优化等级标注。

In [63]:
df = spark.read.csv('training.csv')
df.show(10)

+---+---+---+---+---+---+---+---+---+---+----+
|_c0|_c1|_c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|
+---+---+---+---+---+---+---+---+---+---+----+
|  H| 10|  H|  J|  H|  Q|  H|  K|  H|  1|   9|
|  S|  J|  S|  K|  S|  Q|  S| 10|  S|  1|   9|
|  D|  Q|  D|  J|  D| 10|  D|  K|  D|  1|   9|
|  C| 10|  C|  J|  C|  K|  C|  1|  C|  Q|   9|
|  C|  1|  C|  K|  C|  J|  C|  Q|  C| 10|   9|
|  H|  2|  H|  4|  H|  3|  H|  5|  H|  6|   8|
|  H|  9|  H|  Q|  H|  J|  H| 10|  H|  K|   8|
|  S|  1|  S|  2|  S|  4|  S|  3|  S|  5|   8|
|  D|  5|  D|  6|  D|  7|  D|  9|  D|  8|   8|
|  C|  1|  C|  4|  C|  3|  C|  2|  C|  5|   8|
+---+---+---+---+---+---+---+---+---+---+----+
only showing top 10 rows



In [64]:
#为保证数据都是数值型，将字符做转换
dict0 = {'1':'A'} 
df = df.replace(dict0)
dict1 = {'H':'3','S':'4','D':'2','C':'1','J':'11','Q':'12','K':'13','A':'14'} 
df = df.replace(dict1)
df.show(10)

+---+---+---+---+---+---+---+---+---+---+----+
|_c0|_c1|_c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|
+---+---+---+---+---+---+---+---+---+---+----+
|  3| 10|  3| 11|  3| 12|  3| 13|  3| 14|   9|
|  4| 11|  4| 13|  4| 12|  4| 10|  4| 14|   9|
|  2| 12|  2| 11|  2| 10|  2| 13|  2| 14|   9|
|  1| 10|  1| 11|  1| 13|  1| 14|  1| 12|   9|
|  1| 14|  1| 13|  1| 11|  1| 12|  1| 10|   9|
|  3|  2|  3|  4|  3|  3|  3|  5|  3|  6|   8|
|  3|  9|  3| 12|  3| 11|  3| 10|  3| 13|   8|
|  4| 14|  4|  2|  4|  4|  4|  3|  4|  5|   8|
|  2|  5|  2|  6|  2|  7|  2|  9|  2|  8|   8|
|  1| 14|  1|  4|  1|  3|  1|  2|  1|  5|   8|
+---+---+---+---+---+---+---+---+---+---+----+
only showing top 10 rows



In [97]:
datas = df.select(df._c0.cast('int'), df._c1.cast('int'), df._c2.cast('int'), df._c3.cast('int'),
                 df._c4.cast('int'),df._c5.cast('int'),df._c6.cast('int'),df._c7.cast('int'),
                 df._c8.cast('int'),df._c9.cast('int'),df._c10.cast('int'))

#通过select函数可以实现datas的列操作
type(datas)

pyspark.sql.dataframe.DataFrame

In [98]:
#转化成pandas.dataframe.
#pyspark.sql.dataframe.DataFrame --> pandas.core.dataFrame --> spark.createDateframe
datas = datas.toPandas()

In [99]:
import pandas as pd
#去重求数方法，如f([1,2,3,4,4]) = f([1,2,3,4]) = 4
def notAlong(a):
    l=[]
    for i in set(a):
        l.append(a.count(i))
    return len(set(a)) - l.count(1)
#重复出现最大次
def maxrepeat(b):
    n=[]
    for i in set(b):
        n.append(b.count(i))
    return max(n)

temp = datas
#特征工程
#处理花色
temp1 = datas[['_c0', '_c2','_c4', '_c6', '_c8']]
max_repeat_time = temp1.apply(lambda x: maxrepeat([i for i in np.array(x)]),axis = 1)
#处理点数
temp2 = datas[['_c1', '_c3','_c5', '_c7', '_c9']]
temp2 = temp2.astype(float)
#一组数的均值
meanPoint = temp2.apply(np.mean, axis = 1)
#一组数的方差
varPoint = temp2.apply(np.var, axis = 1)
setLastPoint = temp2.apply(lambda x: len(set([i for i in np.array(x)])),axis=1)
notLPoint = temp2.apply(lambda x: notAlong([i for i  in np.array(x)]),axis=1)

da = {'a':meanPoint, 'b':varPoint, 'c':setLastPoint, 'd':notLPoint,'e':max_repeat_time}
frame = pd.DataFrame(da)
frame['c10'] = temp['_c10']
datas = spark.createDataFrame(frame)

In [102]:
datas.columns

['a', 'b', 'c', 'd', 'e', 'c10']

In [105]:
from pyspark.ml.feature import VectorAssembler
ins = datas.columns
ins.remove('c10')
assember = VectorAssembler(inputCols = ins,outputCol = 'features')
output = assember.transform(datas)
label_features = output.select('features','c10').toDF('features', 'label')
label_features.show(5, truncate=False)

+----------------------+-----+
|features              |label|
+----------------------+-----+
|[12.0,2.0,5.0,0.0,5.0]|9    |
|[12.0,2.0,5.0,0.0,5.0]|9    |
|[12.0,2.0,5.0,0.0,5.0]|9    |
|[12.0,2.0,5.0,0.0,5.0]|9    |
|[12.0,2.0,5.0,0.0,5.0]|9    |
+----------------------+-----+
only showing top 5 rows



In [106]:
from pyspark.ml.classification import DecisionTreeClassifier
model = DecisionTreeClassifier()

(trainingData, testData) = label_features.randomSplit([0.8, 0.2])
model = model.fit(trainingData)
predictions=model.transform(testData)
predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[3.6,1.8400000000...|    3|[0.0,0.0,0.0,8498...|[0.0,0.0,0.0,1.0,...|       3.0|
|[3.8,3.7600000000...|    3|[0.0,0.0,0.0,8498...|[0.0,0.0,0.0,1.0,...|       3.0|
|[3.8,4.96,3.0,1.0...|   14|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|      14.0|
|[4.0,1.2,3.0,2.0,...|    2|[0.0,0.0,956.0,0....|[0.0,0.0,1.0,0.0,...|       2.0|
|[4.0,2.8,4.0,1.0,...|    3|[0.0,0.0,0.0,8498...|[0.0,0.0,0.0,1.0,...|       3.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [107]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
            metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
accuracy

0.9996005592170961